In [3]:
import transformers
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM

model_name_or_path = "/home/jye/huggingface/pretrained_model/Meta-Llama-3-8B-Instruct/"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import transformers
import torch
import os
from modelscope import snapshot_download
model_id = snapshot_download("LLM-Research/Meta-Llama-3-8B-Instruct")
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device="cuda:6",
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
)

terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

outputs = pipeline(
        prompt,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=False,
        temperature=0.6,
        top_p=1,
        repetition_penalty=1.05
)
print(outputs[0]["generated_text"][len(prompt):])


In [ ]:
outputs

In [ ]:
from vllm import LLM, SamplingParams
import torch

# model_id = "/home/jye/huggingface/pretrained_model/Meta-Llama-3-8B-Instruct"

llm = LLM(
    model=model_id,
    tensor_parallel_size=2,
    dtype=torch.float16,
)

tokenizer = llm.get_tokenizer()

conversations = tokenizer.apply_chat_template(
    [{'role':'user', 'content':'Hello!'}],
    tokenize=False,
)

outputs = llm.generate(
    [conversations],
    SamplingParams(
        max_length=1048,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.05,
        stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>"), 128001, 128009]
    )
)

In [ ]:
import json

# read jsonl 
with open('/home/jye/learn/LLM-Factory/data/IEMOCAP_0_8.jsonl') as f:
    data = [json.loads(line) for line in f]

In [6]:
from datasets import load_dataset, load_from_disk

local_dataset = load_from_disk("/home/jye/huggingface/datasets/Universal-NER/Pile-NER-type")


In [7]:
print(local_dataset['train'][0])

{'id': 'ner_0', 'conversations': [{'from': 'human', 'value': 'Text: Q:\n\nPosition character based on enemy coordinates in lua\n\nI have written a function here which should turn my character based on enemy coordinates but it\'s not perfect because it does not always turn where I want it to and perhaps there is a better way of writing it\nlocal myPosition = {x = 350, y = 355}\nlocal enemyPosition = {x = 352, y = 354}\nlocal xValue, yValue, xDir, yDir, dir\n\nif myPosition.x > enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelseif myPosition.x < enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelse\n    xValue = 0\nend\n\nif myPosition.y > enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelseif myPosition.y < enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelse\n    yValue = 0\nend\n\nif xValue < 0 then\n    xDir = "TURN RIGHT"\nelseif xValue > 0 then\n    xDir = "TURN LEFT"\nend\n\nif yValue < 0 then\n    yDir = 